In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Dict
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage
from typing import List
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from IPython.display import Image, display
import json
import re
import textwrap

In [ ]:
load_dotenv()
GROQ_API_KEY=os.getenv("groq_api_key")
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
llm=ChatGroq(model_name="Gemma2-9b-It")

### Defining Agents functions

In [ ]:
# Load the prompt from an external file
with open("prompt.txt", "r", encoding="utf-8") as file:
    SALES_EXECUTIVE_PROMPT = file.read()

In [ ]:
def sales_agent(messages: List[dict]) -> dict:

    system_message = {
        "role": "system",
        "content": SALES_EXECUTIVE_PROMPT,
    }

    messages = [system_message] + messages 

    response: AIMessage = llm.invoke(messages)

    return {
        "role": "assistant",
        "content": response.content
    }

In [ ]:
# Simulated user

system_prompt_template = """You are a customer. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Carryminati. You want to buy a bike. \
You want to inquire about it. \
"""

prompt = prompt.partial(name="Carryminati", instructions=instructions)

simulated_user = prompt | llm

In [ ]:
def evaluate_response(simulated_user_message:str, chatbot_message:str) -> Dict:
    """Evaluate the Sales Agent's response based on key metrics."""
    
    evaluation_prompt = f"""
    You are an Evaluator AI that assesses responses from a Sales Agent based on company policies.

    User Query: {simulated_user_message}
    Sales Agent Response: {chatbot_message}

    Evaluate the response on:
    - Accuracy (1-5): Did the agent provide accurate information without errors?
    - Completeness (1-5): Did the agent cover all necessary details and follow all provided rules?
    - Compliance (1-5): Did the agent follow company policies (e.g., not providing prices, referring to the correct contacts)?
    - Engagement (1-5): Did the agent engage with the customer in a friendly and helpful manner?

    Provide a short feedback summary.

    Output as JSON:
    {{
        "accuracy": int,
        "completeness": int,
        "compliance": int,
        "engagement": int,
        "feedback": "string"
    }}
    """
    
    eval_response = llm.invoke([SystemMessage(content=evaluation_prompt)])
    return eval_response.content

### Defining Nodes

In [ ]:
def sales_agent_node(state):

    messages = state["messages"]
    messages = [convert_message_to_dict(m) for m in messages]

    chat_bot_response = sales_agent(messages)
    chat_message = AIMessage(content=chat_bot_response["content"])
    
    last_user_message = messages[-1]["content"]
    evaluation = evaluate_response(last_user_message, chat_message.content)
    
    return {
        "messages": [chat_message],
        "evaluation": evaluation
    }

In [ ]:
def _swap_roles(messages):

    new_messages = [] 

    for m in messages:
        if isinstance(m, AIMessage):  
            new_messages.append(HumanMessage(content=m.content))  # AI → Human
        else:
            new_messages.append(AIMessage(content=m.content))  # Human → AI
    
    return new_messages

In [ ]:
def simulated_user_node(state):

    messages = state["messages"] 

    new_messages = _swap_roles(messages) 

    response = simulated_user.invoke({"messages": new_messages})  

    return {
        "messages": [HumanMessage(content=response.content)]  
    }

In [ ]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 8:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

### Making graph structure

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    evaluation: dict 

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", sales_agent_node)
graph_builder.add_edge("user", "chat_bot")
graph_builder.add_conditional_edges(
    "chat_bot",
    should_continue,
    {
        "end": END,
        "continue": "user",
    },
)
graph_builder.add_edge(START, "user")
simulation = graph_builder.compile()

In [ ]:
try:
    display(Image(simulation.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

### Simulating user queries

In [ ]:
def serialize_message(message):
    """Convert message objects into dictionaries for JSON serialization."""
    if isinstance(message, (list, tuple)):
        return [serialize_message(m) for m in message]
    if hasattr(message, "__dict__"):
        return message.__dict__
    return message  

In [ ]:
for chunk in simulation.stream({"messages": [], "evaluation": {}}):
    if END not in chunk:
        chunk_serializable = {}
        for key, value in chunk.items():
            if isinstance(value, dict) and 'messages' in value:
                chunk_serializable[key] = {
                    'messages': [{
                        'content': msg.content,
                        'type': msg.__class__.__name__
                    } for msg in value['messages']]
                }
            else:
                chunk_serializable[key] = value
        print(json.dumps(chunk_serializable, indent=4)) 
        
        evaluation_raw = chunk.get("chat_bot", {}).get("evaluation")
        if evaluation_raw:
            try:
                evaluation_cleaned = re.sub(r"^```json\n|\n```$", "", evaluation_raw.strip())

                evaluation_data = json.loads(evaluation_cleaned)
                
                print("Evaluation:")
                for key, value in evaluation_data.items():
                    if isinstance(value, str):
                        print(f"{key.capitalize()}:")
                        print(textwrap.fill(value, width=80)) 
                    else:
                        print(f"{key.capitalize()}: {value}")

            except json.JSONDecodeError as e:
                print("Error: Invalid JSON in evaluation", e)

        print("----")


### Asking custom queries

In [ ]:
def ask_custom_question(user_input: str):
    messages = [{"role": "user", "content": user_input}]

    chat_bot_response = sales_agent(messages)
    chat_message = AIMessage(content=chat_bot_response["content"])

    evaluation = evaluate_response(user_input, chat_message.content)

    return {
        "chatbot_response": chat_message.content,
        "evaluation": evaluation
    }

In [ ]:
custom_question = "Can you tell me price of bike Speed Twin 1200?"
response_data = ask_custom_question(custom_question)

In [ ]:
print("Chatbot Response:")
print(response_data["chatbot_response"])
print("\nEvaluation:")
print(response_data["evaluation"])